In [ ]:
#video prediction

In [ ]:
import cv2
import os

def extract_frames(video_path, frame_count=5, resize_dims=(128, 128)):
    # Extract video file name without extension
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_capture = sorted(set(int(total_frames * i / frame_count) for i in range(frame_count)))

    extracted_frames = []
    frame_idx = 0
    saved_frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx in frames_to_capture:
            saved_frame_count += 1
            # Resize frame
            resized_frame = cv2.resize(frame, resize_dims)
            # Save frame as image, named according to the video and frame number
            frame_filename = f'{video_name}_frame{saved_frame_count}.jpg'
            cv2.imwrite(frame_filename, resized_frame)
            extracted_frames.append(frame_filename)
        frame_idx += 1
        if saved_frame_count == frame_count:
            break

    cap.release()
    return extracted_frames

# Example usage:
video_path = 'video.mp4'  # Replace with your actual video file path
frames = extract_frames(video_path)

In [ ]:
#!pip install ultralytics==8.0.230
from ultralytics import YOLO

# Load the best version of the model
# This must be the location to the model
model = YOLO('Trained_Model.pt')


# Select image or list of images to predict
results = model(frames, save=True, save_txt=True, project="predictions")


0: 128x128 surprise 0.64, embarrassment 0.22, confusion 0.13, frustration 0.02, 1: 128x128 surprise 0.47, embarrassment 0.38, confusion 0.14, frustration 0.01, 2: 128x128 surprise 0.58, embarrassment 0.24, confusion 0.17, frustration 0.02, 3: 128x128 surprise 0.38, embarrassment 0.33, confusion 0.28, frustration 0.01, 4: 128x128 surprise 0.53, confusion 0.26, embarrassment 0.20, frustration 0.01, 251.4ms
Speed: 0.7ms preprocess, 50.3ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)
Results saved to predictions/predict
5 labels saved to predictions/predict/labels


In [ ]:
#audio prediction

In [ ]:
import librosa
from keras.models import load_model
from moviepy.editor import VideoFileClip

In [ ]:
model_path = 'my_lstm_model.h5'
from keras.models import load_model
model = load_model(model_path)

In [ ]:
from moviepy.editor import VideoFileClip

# Path to your video file
#video_path = 'Confusion-Test.mp4'

# Create a VideoFileClip instance
video_clip = VideoFileClip(video_path)

# Set the path for the extracted audio file
audio_path = 'extracted_audio.wav'

# Extract the audio from the video clip and save it to the audio_path
video_clip.audio.write_audiofile(audio_path)

# The audio is now extracted and saved as 'extracted_audio.wav'


MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


In [ ]:
import librosa
import numpy as np

data, sample_rate = librosa.load(audio_path, sr=None)
mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
mfccs = np.mean(mfccs.T, axis=0)
mfccs = np.expand_dims(mfccs, axis=0)
mfccs = np.expand_dims(mfccs, axis=2)
prediction = model.predict(mfccs)
predicted_class = np.argmax(prediction, axis=1)
prediction_score = np.max(prediction, axis=1)[0]  # Max score for the predicted class
emotion_labels = ['frustration', 'confusion', 'embarrassment', 'surprise']  # Adjusted labels
predicted_emotion = emotion_labels[predicted_class[0]]

# Specify the full path where the audio.txt file should be saved
file_path = 'predictions/predict/labels/audio.txt'

# Writing the score and emotion to a txt file in the requested format
with open(file_path, 'w') as file:
    file.write(f'{prediction_score:.2f} {predicted_emotion}')

1/1 [==============================] - 0s 474ms/step


In [ ]:
#fusion

In [ ]:
import os

emotion_to_emojis = {
    'confusion': ('🤨', '🤔', '😵‍💫'),
    'embarrassment': ('😬', '😥', '🙈'),
    'frustration': ('😣', '😐', '😤'),
    'surprise': ('😲', '😳', '😱')
}

def get_emoji(score, emojis):
    if 0.0 <= score <= 0.33:
        return emojis[0]
    elif 0.34 <= score <= 0.66:
        return emojis[1]
    elif 0.67 <= score <= 1.0:
        return emojis[2]
    return None

# Folder Path
folder_path = 'predictions/predict/labels/'

# Initialize variables to store the maximum score and corresponding emotion
max_score = -1
max_emotion = None
selected_emoji = None

# Iterate over each file in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            parts = line.split()
            score = float(parts[0])
            emotion = parts[1].lower()

            if score > max_score:
                max_score = score
                max_emotion = emotion
                selected_emoji = get_emoji(score, emotion_to_emojis[emotion]) if emotion in emotion_to_emojis else None

if selected_emoji:
    print(f"Emoji: ({max_emotion.capitalize()} score: {max_score}): {selected_emoji}")
else:
    print("No valid emotion found.")

Emoji: (Surprise score: 0.64): 😳
